# Class 매핑 및 YAML 생성

Coarse/Fine 클래스 ID 매핑 및 YOLO names.yaml 생성

**원본 스크립트**: `aws_icon_class_mapping_and_yaml.sh`


## 개요
AWS 아이콘 coarse/fine 클래스 ID 매핑 + YOLO names yaml 생성

### 사용 목적
- taxonomy_coarse.csv / taxonomy_fine.csv를 기준으로
  - coarse/fine 클래스에 안정적인 ID를 부여
  - class_mapping_*.json 생성 (id<->name 양방향 매핑)
  - yolo_*_names.yaml 생성 (YOLO dataset.yaml에서 names로 재사용)

### 사용법
```bash
./aws_icon_class_mapping_and_yaml.sh ./dataset/icons
```

### 인자
- `$1`: taxonomy CSV 파일이 들어있는 디렉터리 (기본값: ./dataset/icons)

### 출력 파일
- class_mapping_coarse.json
- class_mapping_fine.json
- yolo_coarse_names.yaml
- yolo_fine_names.yaml

### 내부 동작 개요
1. taxonomy_coarse.csv 읽어서 coarse_class를 정렬 후 ID 부여
2. taxonomy_fine.csv 읽어서 canonical_service_name을 알파벳 순으로 정렬 후 ID 부여
3. coarse/fine 각각에 대해 id_to_name / name_to_id 구조의 JSON 저장 및 YOLO names 형식의 yaml 저장


In [ ]:
set -euo pipefail
DATA_DIR="${1:-./dataset/icons}"
export DATA_DIR
if [ ! -d "$DATA_DIR" ]; then
  echo "ERROR: DATA_DIR '$DATA_DIR' 디렉터리가 없습니다." >&2
  exit 1
fi
echo "[INFO] DATA_DIR = $DATA_DIR"
echo "[INFO] taxonomy_coarse.csv / taxonomy_fine.csv 기준으로 클래스 매핑을 생성합니다."


In [ ]:
import os
import json
from pathlib import Path

import pandas as pd

DATA_DIR = os.environ["DATA_DIR"]
data_path = Path(DATA_DIR)

tax_coarse_path = data_path / "taxonomy_coarse.csv"
tax_fine_path = data_path / "taxonomy_fine.csv"

if not tax_coarse_path.exists():
    raise FileNotFoundError(f"{tax_coarse_path} not found")
if not tax_fine_path.exists():
    raise FileNotFoundError(f"{tax_fine_path} not found")

print(f"[INFO] Load: {tax_coarse_path}")
df_coarse = pd.read_csv(tax_coarse_path)

print(f"[INFO] Load: {tax_fine_path}")
df_fine = pd.read_csv(tax_fine_path)

# ============================================
# 1. coarse 클래스 ID 매핑
# ============================================

# coarse_class 이름 기준 정렬
coarse_names = sorted(df_coarse["coarse_class"].unique())
id_to_coarse = {i: name for i, name in enumerate(coarse_names)}
coarse_to_id = {name: i for i, name in id_to_coarse.items()}

coarse_mapping = {
    "id_to_coarse": {str(k): v for k, v in id_to_coarse.items()},
    "coarse_to_id": coarse_to_id,
}

coarse_json_path = data_path / "class_mapping_coarse.json"
coarse_json_path.write_text(
    json.dumps(coarse_mapping, indent=2, ensure_ascii=False),
    encoding="utf-8",
)
print(f"[OK] class_mapping_coarse.json 저장: {coarse_json_path}")

# YOLO names용 yaml (coarse)
# 형식:
#   names:
#     0: Analytics
#     1: ApplicationIntegration
#     ...
yolo_coarse_yaml = ["names:"]
for idx, name in id_to_coarse.items():
    # 인덴트 2칸
    yolo_coarse_yaml.append(f"  {idx}: {name}")

yolo_coarse_path = data_path / "yolo_coarse_names.yaml"
yolo_coarse_path.write_text("\n".join(yolo_coarse_yaml), encoding="utf-8")
print(f"[OK] yolo_coarse_names.yaml 저장: {yolo_coarse_path}")

# ============================================
# 2. fine 클래스 ID 매핑
# ============================================

# canonical_service_name 알파벳 기준 정렬
fine_names = sorted(df_fine["canonical_service_name"].unique())
id_to_fine = {i: name for i, name in enumerate(fine_names)}
fine_to_id = {name: i for i, name in id_to_fine.items()}

fine_mapping = {
    "id_to_fine": {str(k): v for k, v in id_to_fine.items()},
    "fine_to_id": fine_to_id,
}

fine_json_path = data_path / "class_mapping_fine.json"
fine_json_path.write_text(
    json.dumps(fine_mapping, indent=2, ensure_ascii=False),
    encoding="utf-8",
)
print(f"[OK] class_mapping_fine.json 저장: {fine_json_path}")

# YOLO names용 yaml (fine)
# 형식:
#   names:
#     0: amazon api gateway
#     1: amazon aurora
#     ...
yolo_fine_yaml = ["names:"]
for idx, name in id_to_fine.items():
    yolo_fine_yaml.append(f"  {idx}: {name}")

yolo_fine_path = data_path / "yolo_fine_names.yaml"
yolo_fine_path.write_text("\n".join(yolo_fine_yaml), encoding="utf-8")
print(f"[OK] yolo_fine_names.yaml 저장: {yolo_fine_path}")

# ============================================
# 3. 요약 출력
# ============================================

print(f"[SUMMARY] coarse classes: {len(coarse_names)}개")
print(f"[SUMMARY] fine services: {len(fine_names)}개")
print("[DONE] coarse/fine 클래스 ID 매핑 + YOLO names yaml 생성 완료.")
PY

